Step 1: Download data files 
Preferred script: download_data_ooi1_0.py

Usage:
1- compare the QC Database to the OOI GUI data catalog, 
2- build netCDF data request urls, and 
3- send requests if prompted.
To Run:
Input variables:
sDir: directory where outputs are saved
username: OOI API username
Token: OOI API password

Module Functions: in Tools (GitHub)
Download_data_nc_interactive.py (Main Script)  
data_request_urls
send_data_requests_nc
data_request_tools not in use 
Interactive_inputs
Prompt usage:
select the platform, node, instruments and methods
start downloading data

In [19]:
import datetime as dt
import pandas as pd
import os
import itertools
import functions.common as cf
from scripts import interactive_inputs
from scripts import data_request_urls_ooi1_0
from scripts import send_data_requests_nc

In [14]:
sDir = '/Users/leila/Documents/Desktop/8thEGOMeeting-notebooks'
f = ''  # optional i.e. 'data_download.csv'
username = 'OOIAPI-4J3Y5ISGDUPDB7'
token = '4OT8YQUME0XOQF'

cf.create_dir(sDir)
now = dt.datetime.now().strftime('%Y%m%dT%H%M')

In [15]:
help(interactive_inputs)

Help on module scripts.interactive_inputs in scripts:

NAME
    scripts.interactive_inputs - Created  on Sep  2018

DESCRIPTION
    @author: Lori Garzio
    @brief: provides interactive inputs for data download

FUNCTIONS
    return_interactive_inputs()

FILE
    /Users/leila/anaconda3/lib/python3.7/site-packages/scripts/interactive_inputs.py




In [18]:
if not f:
    array, subsite, node, inst, delivery_methods = interactive_inputs.return_interactive_inputs()
    f_url_list = scripts.data_request_urls_ooi1_0.main(sDir, array, subsite, node, inst, delivery_methods, now)

These arrays are listed in the QC Database:
['CE', 'CP', 'GA', 'GI', 'GP', 'GS', 'RS']

Please select arrays. Must be comma separated (if choosing multiple) or press enter to select all: CP
These subsites are listed in the QC Database for the selected array(s):
['CP01CNPM', 'CP01CNSM', 'CP01CNSP', 'CP02PMCI', 'CP02PMCO', 'CP02PMUI', 'CP02PMUO', 'CP03ISPM', 'CP03ISSM', 'CP03ISSP', 'CP04OSPM', 'CP04OSSM', 'CP05MOAS']

Please select fully-qualified subsites. Must be comma separated (if choosing multiple) or press enter to select all: CP05MOAS

These nodes are listed in the QC Database for the selected subsite(s):
['A6263', 'A6264', 'GL335', 'GL336', 'GL339', 'GL340', 'GL374', 'GL375', 'GL376', 'GL379', 'GL380', 'GL387', 'GL388', 'GL389', 'PG564', 'PG583']

Please select fully-qualified nodes. Must be comma separated (if choosing multiple) or press enter to select all: GL335

These instruments are listed in the QC Database for the selected array(s), subsite(s), and node(s):
['ADCPA', 'CTDG

In [20]:
thredds_output_urls = send_data_requests_nc.main(sDir, f_url_list, username, token, now)


There are 8 requests to send, are you sure you want to continue? y/<n>: y

Request url 1 of 8: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL335/03-CTDGVM000/recovered_host/ctdgv_m_glider_instrument_recovered?beginDT=2014-10-06T20:16:00.000Z&endDT=2017-03-06T22:45:00.000Z&include_annotations=true&include_provenance=true
Data request sent
Data available for request

Request url 2 of 8: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL335/03-CTDGVM000/telemetered/ctdgv_m_glider_instrument?beginDT=2014-10-06T20:16:00.000Z&endDT=2017-03-06T22:45:00.000Z&include_annotations=true&include_provenance=true
Data request sent
Data available for request

Request url 3 of 8: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL335/04-DOSTAM000/recovered_host/dosta_abcdjm_glider_recovered?beginDT=2014-10-06T20:16:00.000Z&endDT=2017-03-06T22:45:00.000Z&include_annotations=true&include_provenance=true
Data request sent
Data ava

In [ ]:
if not f:
    array, subsite, node, inst, delivery_methods = scripts.interactive_inputs.return_interactive_inputs()
    f_url_list = scripts.data_request_urls_ooi1_0.main(sDir, array, subsite, node, inst, delivery_methods, now)
else:
    df = pd.read_csv(os.path.join(sDir, f))
    url_list = []
    for i, j in df.iterrows():
        array = scripts.data_request_tools.check_str(j['array'])
        array = scripts.data_request_tools.format_inputs(array)
        refdes = j['reference_designator']
        if type(refdes) == str:
            subsite = scripts.data_request_tools.format_inputs(refdes.split('-')[0])
            node = scripts.data_request_tools.format_inputs(refdes.split('-')[1])
            inst = scripts.data_request_tools.format_inputs('-'.join((refdes.split('-')[2], refdes.split('-')[3])))
        else:
            subsite = scripts.data_request_tools.check_str(j['subsite'])
            subsite = scripts.data_request_tools.format_inputs(subsite)
            node = scripts.data_request_tools.check_str(j['node'])
            node = scripts.data_request_tools.format_inputs(node)
            inst = scripts.data_request_tools.check_str(j['sensor'])
            inst = scripts.data_request_tools.format_inputs(inst)
        delivery_methods = scripts.data_request_tools.check_str(j['delivery_method'])
        delivery_methods = scripts.data_request_tools.format_inputs(delivery_methods)

        urls = scripts.data_request_urls_ooi1_0.main(sDir, array, subsite, node, inst, delivery_methods, now)
        url_list.append(urls)

    f_url_list = list(itertools.chain(*url_list))

thredds_output_urls = scripts.send_data_requests_nc.main(sDir, f_url_list, username, token, now)

print('\nSeeing if the requests have fulfilled...')
for i in range(len(thredds_output_urls)):
    url = thredds_output_urls[i]
    print('\nDataset {} of {}: {}'.format((i + 1), len(thredds_output_urls), url))
    if 'no_output_url' not in url:
        cf.check_request_status(url)